In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

import torch


In [ ]:
from transformers import pipeline
import transformers

# Define the model ID
model_id = "meta-llama/Llama-3.2-3B"

# Set up the pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

# Define the chat messages
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

# Convert messages to prompt format using the pipeline tokenizer
prompt = pipeline.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

# Define the end-of-sequence token IDs for the model
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Generate the response
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

# Print the generated text without the prompt prefix
print(outputs[0]["generated_text"][len(prompt):])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LangChain에서 Hugging Face 모델을 연결하기 위해, 먼저 Transformers 라이브러리로 모델을 로드한 뒤 pipeline 객체를 생성합니다.

In [5]:
# Move model to CUDA device 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set pad_token_id if not already set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Prepare the input text
input_text = "너는 누구니?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True)  # Generate inputs

# Ensure inputs are on the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate text with attention mask
outputs = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],  # Provide attention mask
    max_length=50,
    num_return_sequences=1
)

# Decode and print generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: 너는 누구니? / 너는 누구니? / 너는 누구니? (너는 누구니?) / 너는 누구니? (너는 누구니?) (너는 누구니?)
Title / Romaji: Neoneun


In [12]:
# Move model to CUDA device 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: 너는 누구니? (Who are you?)
너는 누구니?
Korean: 너는 누구니
Hangul: Neoneun Nuguni
Director: Kim Sung-Su, Kang Hyung-Chul
Writer: Yoo Seong-Ho
Networks: SBS, iTV
A series about the 20


In [36]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 민봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. "),
    ("user", "{input}")
])

chain = prompt | llama_chain
chain.invoke({"input": "LLM이 뭐야?"})

'System: You are a helpful, professional assistant named 민봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. \nHuman: LLM이 뭐야? AI야? AI야? AI야'

In [35]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llama_chain| StrOutputParser()

chain.invoke({"input": "대한민국 수도가 어디야?"})

'System: You are a helpful, professional assistant named 민봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. \nHuman: 대한민국 수도가 어디야? \nSystem: 대한민국 수도는 서울특별시입니다'

In [21]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [22]:
# 사용할 Few-shot예제를 정의합니다.
# 여기서도 마찬가지로 뉴스 제목 생성에 관해 진행하겠습니다.
examples = [
    {
        "domain": "경제",
        "answer": "ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다",
    },
    {
        "domain": "정치",
        "answer": "정당 간 연합 논의 본격화, 정치 지형에 미칠 영향은",
    },
    {
        "domain": "과학",
        "answer": "유전자 편집 기술의 미래: CRISPR의 혁신과 윤리적 논의"
    },
]

# 입력할 도메인, 그리고 답변을 PromptTemplate에 입력합니다.
example_prompt = PromptTemplate.from_template(
    "Domain:\n{domain}\nAnswer:\n{answer}"
)
print(example_prompt.format(**examples[0]))

Domain:
경제
Answer:
ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다


In [23]:
### TODO
# example, example_prompt, input_variables를 활용해주세요
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Domain:\n{domain}\nAnswer:",
    input_variables=["domain"],
)
###

domain = "환경"
final_prompt = prompt.format(domain=domain)
print(final_prompt)

Domain:
경제
Answer:
ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다

Domain:
정치
Answer:
정당 간 연합 논의 본격화, 정치 지형에 미칠 영향은

Domain:
과학
Answer:
유전자 편집 기술의 미래: CRISPR의 혁신과 윤리적 논의

Domain:
환경
Answer:


In [25]:
# 결과 출력
answer = llama_chain.invoke(final_prompt)
print(answer)

Domain:
경제
Answer:
ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다

Domain:
정치
Answer:
정당 간 연합 논의 본격화, 정치 지형에 미칠 영향은

Domain:
과학
Answer:
유전자 편집 기술의 미래: CRISPR의 혁신과 윤리적 논의

Domain:
환경
Answer:이번에 새로 나온 신제품입니다.
Dies ist ein neues Produkt.
국내에서 코로나19 백신 접종이 시작된 지 100일을 맞았다. 2일 코로나19 예방접종대응추진단에 따르면 이날 0시 기준 누적 접종자는 1,000만 2,000명이다. 전 국민의 19.5%가 1차 접종을 마쳤고, 2차 접종
